## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 主キー・外部キー

  * `Sales (sale_id, year)` が複合 PK
  * `Sales.product_id` → `Product.product_id` への FK
* 並び順: 任意（`ORDER BY` 不要）
* 判定は ID 基準:

  * `product_id` で結合
* `NOT IN` は不要（単純な内部結合だけで良いケース）

---

## 1) 問題

### PROBLEM_STATEMENT

Sales と Product の 2 テーブルがある。

* **Sales**

  * 各行は、ある年における `product_id` の販売を表す
  * `price` は 1 単位あたりの単価
* **Product**

  * `product_id` ごとの `product_name` を持つ

**各 sale_id に対して**, 対応する `product_name`, `year`, `price` を取得するクエリを書け。
結果の行順は任意。

### TABLES_OR_SCHEMAS

```text
Table: Sales
+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) が PK
product_id は Product.product_id への FK

Table: Product
+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| product_id   | int     |
| product_name | varchar |
+--------------+---------+
product_id が PK
```

### OUTPUT_COLUMNS_AND_RULES

* 出力列

  * `product_name`
  * `year`
  * `price`
* 各行は Sales の 1 レコードに対応
* 結果の並び順は任意

---

## 2) 最適解（単一クエリ）

この問題は

* 「各 Sales 行に対して、Product から名前を引いてくるだけ」
* グループ集計・上位 k 抽出・重複排除のロジックも不要

なので、**CTE やウィンドウ関数は使わず、素直な内部結合が最短・最速**です。

```sql
SELECT
  p.product_name,
  s.year,
  s.price
FROM Sales   AS s
JOIN Product AS p
  ON p.product_id = s.product_id;

Runtime 1178 ms
Beats 65.91%

```

### ポイント

* 必要列だけを投影

  * `sale_id` や `quantity` は要求されていないので出さない
* `INNER JOIN`（省略形 `JOIN`）でProduct に存在しない `product_id` は自動的に除外される
* 並び順指定なし（any order）なので `ORDER BY` も省略し、余計なソートコストをかけない

---

### 代替（構造をテンプレに寄せる CTE 版・性能は同じ）

テンプレ構造を意識して「事前に必要列だけ抜き出す」書き方もできます。
実行計画的にはほぼ同じですが、ロジックを段階的に読ませたいときに使えます。

```sql
WITH pre AS (
  SELECT
    s.product_id,
    s.year,
    s.price
  FROM Sales AS s
)
SELECT
  p.product_name,
  pre.year,
  pre.price
FROM pre
JOIN Product AS p
  ON p.product_id = pre.product_id;

Runtime 1163 ms
Beats 70.73%

```

* `pre` CTE で

  * Sales から必要列だけを抽出
* その結果に対して Product を JOIN

処理内容は最初の解と同じで、PostgreSQL は CTE をインライン化するので **性能差はほぼゼロ** です。

---

## 3) 要点解説

1. **ID 基準の結合がすべて**

   * 問題の本質は「`Sales.product_id` を `Product.product_id` にマッピングする」だけ
   * したがって、`JOIN` 以外の複雑なロジックは不要

2. **ウィンドウ関数・LATERAL は出番なし**

   * ランキング・グループ内上位・累積集計などは一切要求されていない
   * `ROW_NUMBER` / `DENSE_RANK` / `LATERAL` は、「別の問題」で活躍するツールと割り切ってよい

3. **`NOT IN` 問題もなし**

   * 今回は「存在する Product を結合するだけ」なので `NOT IN` 自体が登場しない
   * PostgreSQL では `NOT IN (…NULL…)` が罠になるが、そのリスクもゼロ

4. **並び順任意 → `ORDER BY` を付けない**

   * LeetCode は「any order」と明記 → `ORDER BY` は付けない方が速い
   * ソートは O(N log N) のコストを食うため、不要ならカット

5. **インデックス前提のプラン**

   * 典型的には

     * `Product.product_id` に PK インデックス
     * `Sales.product_id` に FK インデックス
   * これにより、`Sales` の全行を走査しつつ `Product` をインデックスルックアップする形で、ほぼ線形時間で処理可能

---

## 4) 計算量（概算）

`Sales` の件数を **N**, `Product` の件数を **M** とする。

1. **JOIN（インデックスあり）**

   * `Sales` 全行スキャン: O(N)
   * 各行について `Product` へのインデックスルックアップ: O(1) 近似 × N
   * 合計: **O(N)** 近似

2. **追加処理**

   * 集約 (`GROUP BY`)・ウィンドウ・ソート (`ORDER BY`) なし
   * よって、典型的な RDB の結合クエリとしてはほぼ最小限のコスト

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Sales テーブル]
  B[入力 Product テーブル]
  C[結合 product_id で内部結合]
  D[列選択 product_name year price]
  E[出力 結果テーブル]

  A --> C
  B --> C
  C --> D
  D --> E
```

この問題は「**PostgreSQL であっても、変に凝らず素直な JOIN が最適**」という、
良いウォームアップ問題の位置づけです。次のウィンドウ関数系の問題に進む足慣らしとしては完璧なレベル感になっています。
